# Crawling and Preprocessing

In [4]:
import tweepy
import csv
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [5]:
api_key = "9dO4xdinSEjyxtBfjzueGFKeW"
api_key_secret = "OrnvT252rsngPhq5lxdBwPh91sNBtsWzFNhW3S6pN8mHftRhu3"
access_token = "811343210777710592-uBEUOZWtTFAFf120csjmZSAKDUPySmZ"
access_token_secret = "dnUsgu5ALWnKz3EJzeaGMg2oatVBI74MEqtN4g8rQjhaO"

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [2]:
from datetime import date

today = date.today()
print("Today:", today)

def category_tweet_retriever(filename, query):
    csvfile = open(filename, 'w', encoding='utf-8')
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["created_at", "author", "text"])

    for tweet in tweepy.Cursor(api.search_tweets, 
                               q=query, 
                               lang='id', 
                               tweet_mode="extended").items(1000):
        csvwriter.writerow([tweet.created_at, tweet.author.screen_name, tweet.full_text])

Today: 2022-10-17


In [17]:
def user_tweet_retriever(filename, username):
    csvfile = open(filename, 'w', encoding='utf-8')
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["created_at", "tweet"])

    for tweet in api.user_timeline(id=username, count=1000, tweet_mode="extended"):
        csvwriter.writerow([tweet.created_at, tweet.full_text])

In [15]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stem_tweet(tweet):
    # create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # stemming process
    sentence = tweet
    output   = stemmer.stem(sentence)
    return output

In [18]:
df_id = pd.read_csv("stopwords_id.txt", header=None)
stopwords_id = df_id[0].to_list()

df_en = pd.read_csv("stopwords_en.txt", header=None)
stopwords_en = df_en[0].to_list()

def clean_tweet(tweet):    
    import stopwordsiso as stopwords

    process = tweet.lower()
    process = process.split()
    process = [word for word in process if not process in stopwords_en]
    process = [word for word in process if not process in stopwords_id]
    process = " ".join(word for word in process)
    process = re.sub("'", "", process) # to avoid removing contractions in english
    process = re.sub("^rt ", "", process)
    process = re.sub("@[A-Za-z0-9_]+","", process)
    process = re.sub("#","", process)
    process = re.sub(r'http\S+', '', process)
    process = re.sub('[()!?]', ' ', process)
    process = re.sub('\[.*?\]',' ', process)
    process = re.sub("[^a-z0-9]"," ", process)
    return process

In [5]:
def clean_spaces(tweet):
    process = tweet
    process = process.split()
    process = " ".join(word for word in process)
    return process